# Importar dependencias

In [1]:
import pandas as pd
from crewai import Agent, Crew, Task, LLM, Process
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import yaml
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from typing import Type
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
from datetime import datetime
import csv

archivo_csv = "portfolio_historical.csv"



# Ponemos el model a usar

In [2]:
load_dotenv()

llm = LLM(model="gpt-4o-mini")

# Cargamos la config de los agentes en los archivos yaml

In [3]:
files = {
    'agents_portfolio': 'config/agents_portfolio.yaml',
    'tasks_portfolio': 'config/tasks_portfolio.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r', encoding='utf-8') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents_portfolio']
tasks_config = configs['tasks_portfolio']


# Creamos las tools

In [9]:
class ObtenerUltimaTransaccionInput(BaseModel):
    """Input schema for GetActualData tool."""
    archivo_csv: str = Field(..., description="The portfolio historical.")

class ObtenerUltimaTransaccion(BaseTool):
    name: str = "Get Last Transaction"
    description: str = "Obtains the last transaction of the portfolio"
    args_schema = ObtenerUltimaTransaccionInput  

    def _run(self, archivo_csv: str) -> dict:
        try:
            # Leer el archivo CSV
            df = pd.read_csv(archivo_csv)
            
            # Obtener la última fila
            ultima_fila = df.iloc[-1]
            return ultima_fila
        except Exception as e:
            print(f"Error al leer el archivo: {e}")
            return None



class GetActualDataInput(BaseModel):
    """Input schema for GetActualData tool."""
    moneda: str = Field(..., description="The cryptocurrency name to fetch data for. This parameter must always be in lowercase and must not be abbreviated.")

class GetActualData(BaseTool):
    name: str = "Get Crypto Actual Data Tool"
    description: str = "Scrape actual price data for a cryptocurrency."
    args_schema = GetActualDataInput  

    def _run(self, moneda: str) -> dict:
        """
        Fetch the current price of a cryptocurrency from CoinMarketCap.
        """
        class Scraper:
            def __init__(self, moneda: str):
                self.moneda = moneda.lower()
                self.url = f"https://coinmarketcap.com/es/currencies/{self.moneda}/"
                self.options = webdriver.ChromeOptions()
                self.options.add_argument("--headless")
            
            def run(self) -> str:
                driver = webdriver.Chrome(
                    service=Service(ChromeDriverManager().install()),
                    options=self.options
            
                )
                driver.get(self.url)
                try:
                    span = driver.find_element(By.CLASS_NAME, "sc-65e7f566-0.WXGwg.base-text")
                    resultado = span.text
                except Exception as e:
                    resultado = f"Error: {e}"
                driver.quit()
                return resultado

        resultado = Scraper(moneda).run()
        if resultado.startswith("Error:"):
            return {"error": resultado}
        return {"Precio": resultado}

class AddCSVLineInput(BaseModel):
    """
    Input schema for the tool that adds a new line to the CSV.
    
    Attributes:
        fecha (str): Date of the entry in YYYY-MM-DD format.
        moneda (str): Name of the cryptocurrency, must be in lowercase.
        patrimonio (float): Current portfolio value.
        precio (float): Price of the cryptocurrency at the time of the entry.
        cantidad (float): Amount of the cryptocurrency.
        operacion (str): Type of operation (e.g., 'UPDATE').
    """
    fecha: str = Field(..., description="Date of the entry in YYYY-MM-DD format.")
    moneda: str = Field(..., description="Name of the cryptocurrency, must be in lowercase.")
    patrimonio: float = Field(..., description="Current portfolio value.")
    precio: float = Field(..., description="Price of the cryptocurrency at the time of the entry.")
    cantidad: float = Field(..., description="Amount of the cryptocurrency.")
    operacion: str = Field(..., description="Type of operation (e.g., 'UPDATE').")

class AddCSVLine(BaseTool):
    name: str = "Add CSV Line"
    description: str = "Adds a new row to the CSV file."
    args_schema = AddCSVLineInput  

    def _run(
        self, 
        fecha: str,
        moneda: str,
        patrimonio: float,
        precio: float,
        cantidad: float,
        operacion: str
    ) -> str:
        """
        Añade una nueva línea al CSV utilizando los parámetros individuales.
        """
        # Construir el diccionario con los datos
        data = {
            "fecha": fecha,
            "moneda": moneda,
            "patrimonio": patrimonio,
            "precio": precio,
            "cantidad": cantidad,
            "operacion": operacion
        }
        
        return self._add_to_csv(data)
        
    def _add_to_csv(self, data: dict) -> str:
        """
        Método auxiliar que maneja la escritura al CSV
        """
        fieldnames = list(data.keys())
        
        try:
            with open(archivo_csv, mode='a', encoding='utf-8', newline='') as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                f.seek(0, 2)
                if f.tell() == 0:
                    writer.writeheader()
                writer.writerow(data)
        except FileNotFoundError:
            return f"El archivo CSV '{archivo_csv}' no existe. Crea el archivo antes de usar esta herramienta."

        return f"Se ha añadido la línea al CSV '{archivo_csv}' exitosamente."



class UpdatePortfolioInput(BaseModel):
    """ Input schema for the tool that updates the static portfolio (CSV).
    inversion_inicial: Value of the initial investment (already previously declared).
    moneda: Name of the cryptocurrency to be updated.
    precio_actualizado: Current price of the cryptocurrency.
    cantidad_moneda: Total amount of the cryptocurrency that is owned.
    """
    inversion_inicial: float = Field(..., description="Value of the initial investment.") 
    moneda: str = Field(..., description="Name of the cryptocurrency to be updated. This parameter must always be in lowercase and must not be abbreviated.") 
    precio_actualizado: float = Field(..., description="Current price of the cryptocurrency.") 
    cantidad_moneda: float = Field(..., description="Total amount of the cryptocurrency that is owned.")

class UpdatePortfolio(BaseTool):
    name: str = "Update Portfolio"
    description: str = (
        "Updates the CSV (static portfolio) by adding a row with the 'UPDATE' operation, "
        "after calculating the current value and the profit/loss in relation to the initial investment."
    )
    args_schema = UpdatePortfolioInput

    def _run(
        self,
        inversion_inicial: float,
        moneda: str,
        precio_actualizado: float,
        cantidad_moneda: float,
    ) -> str:
        # 1. Calcular el patrimonio actual
        patrimonio_actual = precio_actualizado * cantidad_moneda

        # 2. Calcular la ganancia/pérdida respecto a la inversión inicial
        ganancia_perdida = patrimonio_actual - inversion_inicial

        # 3. Crear una instancia del input para la línea CSV con la operación 'UPDATE'
        nueva_fila = AddCSVLineInput(
            fecha=datetime.now().strftime("%Y-%m-%d"),
            moneda=moneda,
            patrimonio=patrimonio_actual,
            precio=precio_actualizado,
            cantidad=cantidad_moneda,
            operacion="UPDATE"
        )

        # 4. Instanciar la herramienta AddCSVLine y utilizarla para añadir la línea al CSV
        add_csv_line_tool = AddCSVLine()
        resultado_csv = add_csv_line_tool._run(nueva_fila.dict())

        # 5. Retornar un mensaje con la información calculada y el resultado de la operación en el CSV
        resultado = (
            f"=== Actualización de Portafolio ===\n"
            f"Moneda: {moneda}\n"
            f"Patrimonio actual calculado: {patrimonio_actual:.2f}\n"
            f"Inversión inicial: {inversion_inicial:.2f}\n"
            f"Ganancia/Pérdida (W/L): {ganancia_perdida:.2f}\n"
            f"Resultado de la operación CSV: {resultado_csv}"
        )
        return resultado


# Creamos los agentes, las tasks y la crew

In [10]:
agente_datos = Agent(
  config=agents_config['agente_datos'],
  llm=llm,
  tools=[GetActualData(), ObtenerUltimaTransaccion()]
)

agente_updater = Agent(
  config=agents_config['agente_updater'],
  llm=llm,
  tools=[UpdatePortfolio()]
)

agent_operator = Agent(
  config=agents_config['agent_operator'],
  llm=llm,
  tools=[AddCSVLine()]
)


obtain_transaction_and_actual_value = Task(
  config=tasks_config['obtain_transaction_and_actual_value'],
  agent=agente_datos
)

update_portfolio = Task(
  config=tasks_config['update_portfolio'],
  agent=agente_updater
)

compare_and_operate = Task(
  config=tasks_config['compare_and_operate'],
  agent=agent_operator
)

portfolio_crew = Crew(
  agents=[
    agente_datos,
    agente_updater,
    agent_operator
  ],
  tasks=[
    obtain_transaction_and_actual_value,
    update_portfolio,
    compare_and_operate
  ],
  process=Process.sequential,  
  verbose=True
)

# Ejecutamos el codigo

In [ ]:
result = portfolio_crew.kickoff(inputs={'archivo_csv': archivo_csv})
print(result)
